# ODSB-16203: Postback Event Volume Check

Check postback volume for `com.towneers.www`

**Events:** `click_pay_payment`, `click_pay_payment_ad`

**Volume Guidelines:**
- ≥30/day → Sufficient
- 10-30/day → Low but usable
- <10/day → Insufficient

In [ ]:
# Setup
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client(project='moloco-ods')

# Configuration
APP_BUNDLE = 'com.towneers.www'
TARGET_EVENTS = ['click_pay_payment', 'click_pay_payment_ad']
LOOKBACK_DAYS = 7

print(f'App: {APP_BUNDLE}')
print(f'Events: {TARGET_EVENTS}')
print(f'Period: {LOOKBACK_DAYS} days')

In [ ]:
# Daily Event Volume
events_str = ", ".join([f"'{e.lower()}'" for e in TARGET_EVENTS])

query = f"""
SELECT
    DATE(timestamp) AS date_utc,
    event.name AS event_name,
    moloco.attributed,
    COUNT(*) AS event_count,
    COUNT(DISTINCT device.ifa) AS unique_devices
FROM `focal-elf-631.prod_stream_view.pb`
WHERE DATE(timestamp) BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL {LOOKBACK_DAYS} DAY) AND CURRENT_DATE()
    AND app.bundle = '{APP_BUNDLE}'
    AND LOWER(event.name) IN ({events_str})
GROUP BY 1, 2, 3
ORDER BY 1 DESC, 2, 3
"""

df_daily = client.query(query).to_dataframe()
df_daily

In [ ]:
# Summary (Per Event + Attribution Breakdown)
query_total = f"""
SELECT
    event.name AS event_name,
    moloco.attributed AS is_attributed,
    COUNT(*) AS total_events,
    COUNT(DISTINCT device.ifa) AS unique_devices,
    ROUND(COUNT(*) / {LOOKBACK_DAYS}.0, 1) AS avg_daily_events,
    ROUND(COUNT(DISTINCT device.ifa) / {LOOKBACK_DAYS}.0, 1) AS avg_daily_devices
FROM `focal-elf-631.prod_stream_view.pb`
WHERE DATE(timestamp) BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL {LOOKBACK_DAYS} DAY) AND CURRENT_DATE()
    AND app.bundle = '{APP_BUNDLE}'
    AND LOWER(event.name) IN ({events_str})
GROUP BY event.name, moloco.attributed
ORDER BY event.name, moloco.attributed DESC
"""

df_total = client.query(query_total).to_dataframe()

print('=' * 80)
print(f"{'Event':<30} {'Attributed':<12} {'Avg Events/Day':>15} {'Avg Devices/Day':>15}")
print('=' * 80)

for _, row in df_total.iterrows():
    avg_daily = row['avg_daily_events']
    if avg_daily >= 30:
        status = '✅'
    elif avg_daily >= 10:
        status = '⚠️'
    else:
        status = '❌'
    attr_str = 'Yes' if row['is_attributed'] else 'No'
    print(f"{row['event_name']:<30} {attr_str:<12} {avg_daily:>14.1f} {row['avg_daily_devices']:>14.1f}  {status}")

print('=' * 80)
print(f"\n✅ >=30/day (Sufficient)  ⚠️ 10-30/day (Low)  ❌ <10/day (Insufficient)")
print(f"\nNote: For RE campaigns, focus on ATTRIBUTED events for model training.")

In [ ]:
# # (Optional) All events for this app
# query_all = f"""
# SELECT event.name, COUNT(*) AS cnt, ROUND(COUNT(*) / {LOOKBACK_DAYS}.0, 1) AS avg_daily
# FROM `focal-elf-631.prod_stream_view.pb`
# WHERE DATE(timestamp) >= DATE_SUB(CURRENT_DATE(), INTERVAL {LOOKBACK_DAYS} DAY)
#     AND app.bundle = '{APP_BUNDLE}'
# GROUP BY 1 ORDER BY 2 DESC LIMIT 20
# """
# client.query(query_all).to_dataframe()